# **Data Ingestion, Airflow**
Created by [Bagas Prakasa](https://www.linkedin.com/in/bagasprakasa/)

## Prequist:
1. Python 3.6 or later
2. Docker-CE

### **What is Data Ingestion ?**
Data ingestion adalah proses pemindahan data dari satu atau beberapa sumber ke suatu penyimpanan. Data tersebut nantinya akan disimpan dan dianalisis lebih lanjut.

### Perbedaan data ingestion dan data integration
Data integration selangkah lebih rumit dari data ingestion. Hal itu disebabkan dalam proses data ingestion kumpulan data hanya dipindahkan ke lokasi baru. Namun, dalam data integration, kumpulan data tersebut akan dipastikan kesesuaiannya meskipun dari sumber yang berbeda. Dengan begitu, proses menganalisis data bisa dilakukan lebih mudah dan akurat.

### **3 Types of Data Ingestion**

![](https://drive.google.com/uc?export=view&id=1fx_6Fe4USuDUnTemBCa-9_3ZoAqbmO3Z)

1. **Real-time**

Mengumpulkan dan mentransfer data dari sistem secara real-time menggunakan change data capture (CDC). Nantinya, CDC akan terus memantau proses transfer data tanpa mengganggu beban kerja database. Biasanya, perusahaan yang menerapkan jenis ini adalah yang membutuhkan transfer data secara cepat dan real-time. Contohnya adalah perusahaan untuk perdagangan pasar saham

2. **Batch-based**

Proses mengumpulkan dan mentransfer data dalam sebuah kumpulan sesuai dengan interval yang telah terjadwal sebelumnya. Sehingga, pengumpulan data bisa berdasarkan jadwal, peristiwa, atau urutan yang telah disesuaikan. Jenis ini berguna ketika perusahaan perlu mengumpulkan data tertentu setiap hari. Serta tidak memerlukan data untuk mengambil keputusan sewaktu-waktu. 

3. **Lambda architecture-based**

Kombinasi dari dua jenis sebelumnya (real-time dan batch-based). Umumnya, lambda architecture-based terdiri dari proses pengumpulan, penyajian, dan lapisan kecepatan. Proses pengumpulan dan penyajian melakukan pengindeksan data. Kemudian, proses lapisan kecepatan akan secara instan mengindeks data yang belum terambil dari proses pengindeksan yang pertama.

### **Why We need Airflow ?**
#### Typical Usecase
Let's discover a typical use case that we will be able to solved with airflow.

For example, we have Data Pipeline to trigger everyday at 00:10 AM below.
![](https://drive.google.com/uc?export=view&id=1xcxsy4VO_fEh9w0I1ZYWXq9elyznolBY)

- We might have the following data pipeline with three tasks: downloading data,processing data and storing data, all of them being executed sequentially.
- All of those tasks might interact with other tools as well. For example, the data might request an API. Then processing data will interact with Spark Job and finally, storing data to do inserts and updates to a database.
- Besides we have to execute those tasks, we also have to make sure that the external tools such as the API or the database are available in order to be sure that the data pipeline will succeed.

The problems that could be happen are?
- The API could be not available (Diagram 2)
- The Spark job failed because out of memory (Diagram 2)
- The database is not available and so we can't doing inserts or updates (Diagram 2)
- And If we have hundreds of different data pipelines to manage at the same time, it would be really hard to manage. (Diagram 3)

![](https://drive.google.com/uc?export=view&id=1hhiI1PkCP_DYmEsaz1cyyUOszD3wsTpi)

![](https://drive.google.com/uc?export=view&id=1IinleZnAdh8lFT1wc5OacNWmtujJJbHN)

So, the reason why we need airflow are: 
- We will be able to manage our data pipelines and execute our tasks in a very reliable way.
- We will be able to monitor and retry our tasks automaticallyif there is any failure.

### What is Airflow ?

Airflow is an open source platform to programmatically author, schedule and monitor workflows.

Another definition, Airflow is an orchestrator allowing you to execute your tasks at the right time, in the right way and in the right order.

#### Benefit of Airflow
![](https://drive.google.com/uc?export=view&id=15uho4htmdaYOObT-ODEyVwbHiAjqvFeU)

- Dynamic : Our data pipeline are dynamic, everything that we can do in Python, we can do it in our data pipelines. So the possibilities are quite limitless at the end and this is truly powerful.
- Scalability : We can execute as many tasks as we want in parallel. Obviously, it depends on our architecture, on our resources. We can execute using Local, Celery or Kubernetes Executor.
- User Interface (UI) : We will be able to monitor our data problems from it, retry our tasks, interact a lot from the user interface of airflow with our data pipelines.
- Extensibility: That means if there is a new tool and we want to interact with that new tool, we don't have to wait for airflow to be upgraded. We can create our own plugin added to our application.

#### Airflow Core Components
![](https://drive.google.com/uc?export=view&id=1GnlZXhwpbCaS4Dp-ctUbNhOPAgdB67VD)

- Web Server : A Flask server with Gunicon serving the user interface.
- Scheduler : A daemon in charge of scheduling tasks and data pipelines.
- Metastore : A database where all the metadata related to Airflow itself, data pipeline, tasks and so on will be stored. Usually we will use PostgreSQL as it is the recommended database to use with airflow. But we can use MySQL or Oracle as long as the database is compatible with SQLAlchemy.
- Executor : Defines how our tasks will be executed, either with Kubernetes, Celery or local executor.
- Worker : The process where the task is executed. The difference between executor and worker is the executor defines how the task should be executed, whereas the worker is actually the process executing the task.

### What Airflow is not ?
![](https://drive.google.com/uc?export=view&id=1hLFypH8mmkfh-XUQWQ3cjjc8-gaFHMOD)

Airflow is not a data processing framework, like Apache Spark. If we have to process terabytes of data, we should not do that in airflow. Because it's definitely not optimized to do this. So if we want to that, we have to use  an operator to send the spark job where the terabytes of data will be processed
inside Spark and not inside airflow.

### Bagaimana Cara Kerja Airflow?

#### One Node Architecture
![](https://drive.google.com/uc?export=view&id=1AdmvD4_bj40E3upc_QCjV-GjC3WegFxF)


Komponen yang ada dalam One Node Architecture Airflow antara lain Webserver, Scheduler, Metastore dan Executor yang berjalan di dalam satu mesin.
 
Cara kerja Arsitektur 1 Node menjalankan sebuah task antara lain:
1. Webserver akan mengambil metadata yang berasal dari Metastore.
2. Scheduler berkomunikasi dengan Metastore dan Executor untuk menjadwalkan task yang perlu dijalankan.
3. Executor yang sudah diberi tahu oleh Scheduler terkait task yang akan di trigger tersebut selanjutnya akan melakukan *push* task ke Queue. 
4. Ketika task sudah dalam Queue, Queue akan menentukan dimana task tersebut akan di eksekusi. Karena Node yang tersedia hanya 1, task tersebut dieksekusi juga dalam Node tersebut.

Arsitektur ini sangat bagus jika digunakan untuk melakukan hal-hal yang bertujuan untuk experimental saja, bukan untuk production.

Jika kita ingin menjalankan task di Airflow untuk tujuan production, kita bisa menggunakan Multi Nodes Architecture (Celery)

#### Multi Nodes Architecture (Celery)

![](https://drive.google.com/uc?export=view&id=1Kt4_kDkWela737dltOGkw-sb-ZqHgHgs)
Dalam Arsitektur Celery, komponen-komponen yang membentuk Airflow terpisah menjadi beberapa Node.

Pada Node 1, terdapat Web server, scheduler dan Executor. Dalam Node 2 terdapat Metastore dan Queue serta ada 3 Worker Node terpisah. Perbedaan antara Arsitektur 1 Node dengan Multiple Node terdapat pada bagian Queue yang terpisah dengan Executor serta adanya Node khusus untuk Worker.

Cara kerja Arsitektur Celery menjalankan sebuah task antara lain:
1. Webserver akan mengambil metadata yang berasal dari Metastore.
2. Scheduler berkomunikasi dengan Metastore dan Executor untuk menjadwalkan task yang perlu dijalankan.
3. Executor yang sudah diberi tahu oleh Scheduler terkait task yang akan di trigger tersebut selanjutnya akan melakukan *push* task ke Queue. 
4. Ketika task sudah dalam Queue, Queue akan menentukan dimana task tersebut akan di eksekusi tergantung Airflow Worker yang sedang idle. 
5. Airflow yang idle akan mengambil task tersebut untuk dieksekusi di dalam mesin Worker.

### Apa itu DAG ?

Sebuah DAG (Directed Acyclic Graph) adalah data pipeline yang ada dalam Airflow, DAG bertugas mengumpulkan Task bersamaan, mengorganisirnya dengan dependencies and relationships untuk menentukan bagaimana Task-task tersebut seharusnya berjalan.

![](https://drive.google.com/uc?export=view&id=1Asz-yz7g4ZzmbFDfo8Chgm_oAr5dKCUS)

- Directed : ada arah panahnya. Misal, A dulu baru B.
- Acyclic : tidak boleh memutar kembali ke job sebelumnya.
- Graph : Sumber dan targetnya bisa banyak.

![](https://drive.google.com/uc?export=view&id=1i8-I7BEUuDt5l2UI19j10ZQpbyjxCvnA)


Gambar diatas menjelaskan bagaimana task di dalam DAG di eksekusi satu per satu sesuai dengan urutan dependency. Dimulai dari T1, kemudian bercabang ke T2 dan T3, kemudian T4 dan terakhir bercabang ke T5 dan T6. DAG yang benar pasti memiliki urutan task yang searah, tidak pernah berulang. Jika berulang, maka DAG akan mengalami error.

### Peran DAG di dalam Airflow

![](https://drive.google.com/uc?export=view&id=14lu61QGeOkhKSPl50S6jtWqnQlgmu1LF)

Dalam ilustrasi ini, kita menggunakan single node. Berikut penjelasan dari diagram diatas:
1. Pertama, kita membuat dag file (python file) yaitu dag.py dan meletakkannya di dalam folder dags Airflow.
2. Webserver dan Scheduler akan melakukan parsing folder dags tersebut untuk mengecek jika ada file dag baru.
3. Ketika new dag terdeteksi dan tertrigger, DagRun object akan langsung terbentuk dan Scheduler mengupdatenya ke Metastore. DagRun adalah instance dari sebuah dag. 
4. DagRun terbentuk dan memiliki status running. Selanjutnya, Task pertama yang ada di dalam data pipeline kita terschedule. 
5. Ketika Task tersebut ready untuk di trigger, maka TaskInstance object akan terbentuk.
6. Selanjutnya, Scheduler akan mengirimkan TaskInstance tersebut ke Executor.
7. TaskInstance running.
8. Executor mengupdate status dari TaskInstance ke Metastore.
9. Ketika TaskInstance success dan completed, Scheduler akan mengecek status TaskInstance tersebut dan akan mengupdatenya di Metastore.
10. Jika semua TaskInstance completed dan status dari DagRun completed juga, Webserver akan melakukan update status dag tersebut menjadi success completed di Airflow UI.


## **OLTP VS OLAP**

### What is OLTP?

An OLTP system **captures and maintains transaction data in a database**. Each transaction involves individual database records made up of multiple fields or columns. Examples include banking and credit card activity or retail checkout scanning.

In OLTP, the emphasis is on fast processing, because OLTP databases are read, written, and updated frequently. If a transaction fails, built-in system logic ensures data integrity.

### What is OLAP?
OLAP applies complex queries to large amounts of historical data, aggregated from OLTP databases and other sources, for **data mining, analytics, and business intelligence projects**. 

In OLAP, the emphasis is on response time to these complex queries. Each query involves one or more columns of data aggregated from many rows. Examples include year-over-year financial performance or marketing lead generation trends. OLAP databases and data warehouses give analysts and decision-makers the ability to use custom reporting tools to turn data into information. Query failure in OLAP does not interrupt or delay transaction processing for customers, but it can delay or impact the accuracy of business intelligence insights.



### **Perbedaan OLAP dan OLTP**

![](https://drive.google.com/uc?export=view&id=1Ravkrc2jzHx4fW1uDiITWaG_2sukNAyL)